In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv(r"C:\Users\HP\Desktop\data\titani1.csv")

In [4]:
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
818,819,0,3,"Holm, Mr. John Fredrik Alexander",male,43.0,0,0,C 7075,6.4500,NaN,S
753,754,0,3,"Jonkoff, Mr. Lalio",male,23.0,0,0,349204,7.8958,NaN,S
587,588,1,1,"Frolicher-Stehli, Mr. Maxmillian",male,60.0,1,1,13567,79.2000,B41,C


In [5]:
dfs = df.drop(columns=['PassengerId','Ticket','Name','Cabin'])

In [6]:
# step1--> train_test_split
x_train,x_test,y_train,y_test = train_test_split(dfs.drop(columns=['Survived']),
                                                dfs['Survived'],
                                                test_size=0.2,
                                                random_state=42)

In [7]:
x_train.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.500,S
733,2,male,23.0,0,0,13.000,S
382,3,male,32.0,0,0,7.925,S


In [8]:
y_train.sample(5)

59     0
530    1
455    1
839    1
394    1
Name: Survived, dtype: int64

In [9]:
     # step2 imputetion transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked ',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [10]:
# step 3 --> One_hot_encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [11]:
# step 4 --> Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [12]:
# step 5 --> Feature Selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [16]:
# step6 ---> train the model
trf5 = DecisionTreeClassifier()

## Create pipeline

In [17]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

### pipelinevvs Make_pipeline
pipeline reqire naming of steps, make pipeline dose not.<br>
(same appies to Column Transformer vs make_column_transformer)

In [26]:
# Alternet syntex
#pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [19]:
from sklearn import set_config
set_config(display='diagram')

In [20]:
# train
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked ',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000019C4BADBB80>)),
                ('trf5', DecisionTreeClassifier())])

## explor the pipe

In [21]:
#code here
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked ',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x0000019C4BADBB80>),
 'trf5': DecisionTreeClassifier()}

In [23]:
# if we want known the steps right this code
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked ',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [32]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [33]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [34]:
# predict
y_predic = pipe.predict(x_test)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predic)

0.6256983240223464

## cross Validation using pipeline

In [36]:
# Cross validetion using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv = 5,scoring='accuracy').mean()

0.6391214419383433

# Gridsearch using pipeline
(hyper parametre tuning)

In [39]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [40]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv = 5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked ',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000019C4BADBB80>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [41]:
grid.best_score_

0.6391214419383433

In [42]:
grid.best_params_

{'trf5__max_depth': 2}

# Exporting the pipeline

In [43]:
# export
import pickle
pickle.dump(pipe,open('pipe.pk1','wb'))